In [1]:
import torch
import torch.nn as nn
import torchvision

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
import os

from torch.utils.data import Dataset, DataLoader
# from torch.utils.data.sampler import Sampler
import torch.optim as optim
import sys
sys.path.append('../')

from dataset import LbpDataset, train_transforms, val_transforms, test_transforms, collate_fn, get_data
from visualize import visualize
from rcnn_model import fasterrcnn_resnet201_fpn, FastRCNNPredictor
from engine import evaluate
import utils
from train_lbp import get_train_test_list

In [2]:
# retinanet anchor
# ((32, 40, 50),
#  (64, 80, 101),
#  (128, 161, 203),
#  (256, 322, 406),
#  (512, 645, 812))

# fasterrcnn anchor
# ((32,), (64,), (128,), (256,), (512,))
# use retinanet anchor 


In [3]:
from torchvision.models.detection.anchor_utils import AnchorGenerator
anchor_sizes = tuple((x, int(x * 2 ** (1.0 / 3)), int(x * 2 ** (2.0 / 3))) for x in [32, 64, 128, 256, 512])
aspect_ratios = ((0.5, 1.0, 2.0),) * len(anchor_sizes)
rpn_anchor_generator = AnchorGenerator(
    anchor_sizes, aspect_ratios
)
print(anchor_sizes)
print(aspect_ratios)

((32, 40, 50), (64, 80, 101), (128, 161, 203), (256, 322, 406), (512, 645, 812))
((0.5, 1.0, 2.0), (0.5, 1.0, 2.0), (0.5, 1.0, 2.0), (0.5, 1.0, 2.0), (0.5, 1.0, 2.0))


In [4]:
from torchvision.models.detection.faster_rcnn import fasterrcnn_resnet50_fpn

In [5]:
import easydict 
args = easydict.EasyDict({ "batch_size": 6, 
                          "epochs": 50, 
                          "data": 0, 
                          'lr':0.1,
                         'momentum':0.9,
                         'weight_decay':1e-4,
                         'start_epoch':0,
                         'gpu':5,
                          'workers':2,
                         'output_dir' :'../trained_model/fasterrcnn_resnet50_fpn_anchor/'})

In [6]:
df = pd.read_csv('../../data/df.csv')
df.head()
# Data loading code
data_dir = '../../data/df.csv'
train_list, test_list = get_train_test_list(data_dir)
train_dataset = LbpDataset(train_list, transform=train_transforms)
test_dataset = LbpDataset(test_list, transform=val_transforms)  

total 4019 train 3014 test 1005
3014
1005


In [7]:
train_sampler = torch.utils.data.RandomSampler(train_dataset)
test_sampler = torch.utils.data.SequentialSampler(test_dataset)

train_loader = DataLoader(
    train_dataset, batch_size=args.batch_size,
    sampler=train_sampler, num_workers=args.workers,
    collate_fn=utils.collate_fn)

test_loader = DataLoader(
    test_dataset, batch_size=args.batch_size,
    sampler=test_sampler, num_workers=args.workers,
    collate_fn=utils.collate_fn)

In [8]:
num_classes = 2
model = fasterrcnn_resnet50_fpn(pretrained=False, min_size=1024, max_size=1024, 
                                rpn_anchor_generator=rpn_anchor_generator)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
device = torch.device('cuda')
model.to(device)
print('model is loaded to gpu')

model is loaded to gpu


In [9]:
# # pretrained.state_dict()
# for p in model.parameters() :
#     if p.requires_grad == True :
#         print(p.shape)

In [10]:
# model.head.regression_head

In [11]:
params = [p for p in model.parameters() if p.requires_grad]
# optimizer = torch.optim.Adam(params, lr=0.0001)
optimizer = torch.optim.SGD(
       params, lr=0.001, momentum=0.9, weight_decay=1e-4)
lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[20, 40, 60, 80, 100], 
                                                    gamma=0.5)

In [12]:
from train_lbp import train_one_epoch

start_time = time.time()
for epoch in range(120):
    train_one_epoch(model, optimizer, train_loader, device, epoch, 600)
    lr_scheduler.step()
    
    if epoch > 20 and epoch % 5 == 0 :
        if args.output_dir:
            checkpoint = {
                'model': model.state_dict(),
                'optimizer': optimizer.state_dict(),
                'lr_scheduler': lr_scheduler.state_dict(),
                'args': args,
                'epoch': epoch
            }
            utils.save_on_master(
                checkpoint,
                os.path.join(args.output_dir, 'model_{}.pth'.format(epoch)))
            utils.save_on_master(
                checkpoint,
                os.path.join(args.output_dir, 'checkpoint.pth'))

        # evaluate after every epoch
        evaluate(model, test_loader, device=device)    
print('total time is {}'.format(time.time() - start_time))    

Epoch: [0]  [  0/503]  eta: 0:16:16  lr: 0.001000  loss: 1.5298 (1.5298)  loss_classifier: 0.8341 (0.8341)  loss_box_reg: 0.0029 (0.0029)  loss_objectness: 0.6651 (0.6651)  loss_rpn_box_reg: 0.0277 (0.0277)  time: 1.9420  data: 1.1181  max mem: 7081
Epoch: [0]  [502/503]  eta: 0:00:00  lr: 0.001000  loss: 0.1414 (0.1914)  loss_classifier: 0.0176 (0.0239)  loss_box_reg: 0.0016 (0.0045)  loss_objectness: 0.1096 (0.1277)  loss_rpn_box_reg: 0.0075 (0.0353)  time: 0.7535  data: 0.0150  max mem: 7349
Epoch: [0] Total time: 0:06:35 (0.7864 s / it)
Epoch: [1]  [  0/503]  eta: 0:15:37  lr: 0.001000  loss: 0.1086 (0.1086)  loss_classifier: 0.0152 (0.0152)  loss_box_reg: 0.0017 (0.0017)  loss_objectness: 0.0860 (0.0860)  loss_rpn_box_reg: 0.0057 (0.0057)  time: 1.8637  data: 1.1166  max mem: 7349
Epoch: [1]  [502/503]  eta: 0:00:00  lr: 0.001000  loss: 0.1339 (0.1617)  loss_classifier: 0.0189 (0.0192)  loss_box_reg: 0.0025 (0.0033)  loss_objectness: 0.1021 (0.1171)  loss_rpn_box_reg: 0.0060 (0.02

In [13]:
evaluate(model, test_loader, device=device) 

creating index...
index created!
Test:  [  0/168]  eta: 0:03:46  model_time: 0.3345 (0.3345)  evaluator_time: 0.0098 (0.0098)  time: 1.3461  data: 0.9761  max mem: 7349
Test:  [100/168]  eta: 0:00:29  model_time: 0.2945 (0.2930)  evaluator_time: 0.0079 (0.0083)  time: 0.4252  data: 0.1125  max mem: 7349
Test:  [167/168]  eta: 0:00:00  model_time: 0.2966 (0.2933)  evaluator_time: 0.0084 (0.0084)  time: 0.4106  data: 0.1029  max mem: 7349
Test: Total time: 0:01:11 (0.4246 s / it)
Averaged stats: model_time: 0.2966 (0.2933)  evaluator_time: 0.0084 (0.0084)
Accumulating evaluation results...
DONE (t=0.22s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.015
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.044
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.006
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=med